In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv(r"C:\Prosenjeet Shil\sem2 practicals\AIML lab\titanic.csv")
print(train.shape)

(891, 12)


In [5]:
#checking for missing data
NAs = pd.concat([train.isnull().sum()], axis=1, keys=["train"])
NAs[NAs.sum(axis=1) > 0]

,train
Age,177
Cabin,687
Embarked,2


In [7]:
train.pop("Cabin")
train.pop("Name")
train.pop("Ticket")

0             A/5 21171
1              PC 17599
2      STON/O2. 3101282
3                113803
4                373450
             ...       
886              211536
887              112053
888          W./C. 6607
889              111369
890              370376
Name: Ticket, Length: 891, dtype: object

In [9]:
train.shape

(891, 9)

In [11]:
train["Age"] = train["Age"].fillna(train["Age"].mean())

In [13]:
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])

In [15]:
train["Pclass"] = train["Pclass"].apply(str)

In [17]:
for col in train.dtypes[train.dtypes == "object"].index:
    for_dummy = train.pop(col)
    train = pd.concat([train,pd.get_dummies(for_dummy,prefix=col)],axis=1)
train.head()

,PassengerId,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,22.0,1,0,7.2500,False,False,True,False,True,False,False,True
1,2,1,38.0,1,0,71.2833,True,False,False,True,False,True,False,False
2,3,1,26.0,0,0,7.9250,False,False,True,True,False,False,False,True
3,4,1,35.0,1,0,53.1000,True,False,False,True,False,False,False,True
4,5,0,35.0,0,0,8.0500,False,False,True,False,True,False,False,True


In [19]:
labels = train.pop("Survived")

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train, labels, test_size=0.25)

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=10)

In [25]:
y_pred = rf.predict(x_test)

In [27]:
from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(y_test, y_pred)
# print(accuracy)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.8161434977578476


In [29]:
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [31]:
#BAGGING (using RandomForestClassifier as base estimator)
bagging_model=BaggingClassifier(estimator=BaggingClassifier(),n_estimators=10)
bagging_model.fit(x_train,y_train)
bagging_predictions=bagging_model.predict(x_test)
bagging_accuracy=accuracy_score(y_test,bagging_predictions)
print("Bagging Accuracy:",bagging_accuracy)

Bagging Accuracy: 0.8430493273542601


In [33]:
#4.Voting
voting_model=VotingClassifier(estimators=[
    ('rf',RandomForestClassifier())
],voting='hard') #'hard' for majority vote,'soft' for weighted average probabilities
voting_model.fit(x_train,y_train)
voting_predictions=voting_model.predict(x_test)
voting_accuracy=accuracy_score(y_test,voting_predictions)
print("Voting Accuracy:",voting_accuracy)

Voting Accuracy: 0.8340807174887892
